In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [3]:
df_raw = pd.read_csv('data_set_da_test.csv')

In [4]:
df_cleaned = df_raw.copy()

df_cleaned['event_ordering'] = df_cleaned.groupby('session')['event_date'].rank(method='first')

df_cleaned['event_date'] = pd.to_datetime(df_cleaned['event_date'])

In [10]:
print(np.min(df_cleaned['event_date'].dt.date))
print(np.max(df_cleaned['event_date'].dt.date))

2022-09-30
2022-10-13


# Main Funnel

In [46]:
df_funnel = df_cleaned.copy()

df_funnel['Awareness'] = np.where(
    df_funnel['event_ordering'] == 1,
    1,
    0
)

df_land_page = df_funnel.loc[
    df_funnel['Awareness'] == 1,
    ['session','page_type']
]


df_funnel['Consideration'] = np.where(
    df_funnel['event_type'] == 'page_view',
    1,
    0
)


df_funnel['Intent'] = np.where(
    df_funnel['event_type'] == 'add_to_cart',
    1,
    0
)


df_funnel['Purchase'] = np.where(
    df_funnel['event_type'] == 'order',
    1,
    0
)


df_funnel = df_funnel[[
    'session',
    'Awareness',
    'Consideration',
    'Intent',	
    'Purchase'	
]]


df_funnel = pd.merge(
    df_funnel,
    df_land_page,
    how='left',
    on='session'
)


df_aware = df_funnel.loc[
    df_funnel['Awareness'] == 1,
    ['session','page_type','Awareness']
].drop_duplicates()

df_consi = df_funnel.loc[
    df_funnel['Consideration'] == 1,
    ['session','page_type','Consideration']
].drop_duplicates()

df_intent = df_funnel.loc[
    df_funnel['Intent'] == 1,
    ['session','page_type','Intent']
].drop_duplicates()

df_pur = df_funnel.loc[
    df_funnel['Purchase'] == 1,
    ['session','page_type','Purchase']
].drop_duplicates()

list_df = [
    df_aware,
    df_consi,
    df_intent,
    df_pur
]


df_funnel = pd.concat(list_df,ignore_index=True).fillna(0)

df_funnel = df_funnel.groupby(by=['session','page_type']).sum().reset_index()

df_funnel = pd.melt(
    df_funnel,
    id_vars=['session','page_type']
)

df_funnel = df_funnel.groupby(by=['page_type','variable']).agg(
    value = ('value','sum')).reset_index()


df_list = []

for page in df_funnel['page_type'].unique():
    df_funnel_loop = df_funnel[df_funnel['page_type']==page]

    base = df_funnel_loop[df_funnel_loop['variable']=='Awareness']['value'].values

    df_funnel_loop['conversion'] = df_funnel_loop['value']/ base

    df_list.append(df_funnel_loop)

df_funnel_final = pd.concat(
    df_list,
    ignore_index=True
)

dict_stage = {
    'Awareness':'1. Awareness',
    'Consideration':'2. Consideration',
    'Intent':'3. Intent',
    'Purchase':'4. Purchase',
}


df_funnel_final['variable'] = df_funnel_final['variable'].map(dict_stage)


df_funnel_final.to_csv('database/main_funnel.csv',index=False)

C:\Users\giovanni.cintra\AppData\Local\Temp\ipykernel_19180\2839663999.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funnel_loop['conversion'] = df_funnel_loop['value']/ base
C:\Users\giovanni.cintra\AppData\Local\Temp\ipykernel_19180\2839663999.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funnel_loop['conversion'] = df_funnel_loop['value']/ base
C:\Users\giovanni.cintra\AppData\Local\Temp\ipykernel_19180\2839663999.py:101: SettingWithCopyWarning: 
A value is trying to be set on a cop

# Average time

In [10]:
df_time = df_cleaned.copy()

df_time = df_time.groupby(by='session').agg(
    first_event_date = ('event_date','min'),
    last_event_date = ('event_date','max')
).reset_index()

df_time['session_time'] = (df_time['last_event_date'] - df_time['first_event_date']).dt.total_seconds()

df_time = pd.merge(
    df_time,
    df_land_page,
    how='left',
    on='session'
)

df_time1 = df_time.groupby(by='page_type').agg(
    session_time_all = ('session_time','mean')).reset_index()

df_time2 = df_time[df_time['session_time']>0].groupby(by='page_type').agg(
    session_time_high_0 = ('session_time','mean')).reset_index()

df_time_final = pd.concat(
    [
        df_time1,
        df_time2
    ],ignore_index=True
).groupby(by='page_type').sum().reset_index()

df_time_final.to_csv('database/average_time.csv',index=False)
# print(df_time[df_time['session_time']>0].groupby(by='page_type')['session_time'].mean())

# Trafic

In [11]:
df_trafic = df_cleaned.copy()

df_trafic = df_trafic[[
    'session',
    'user'
]].drop_duplicates()

df_trafic = pd.merge(
    df_trafic,
    df_land_page,
    how='left',
    on='session'
)


df_trafic1 = df_trafic.groupby(by='page_type')['user'].nunique().reset_index()
df_trafic2 = df_trafic.groupby(by='page_type')['user'].size().reset_index()

df_trafic1.columns = ['page_type','unique_users']
df_trafic2.columns = ['page_type','total_users']

df_trafic_final = pd.concat(
    [
        df_trafic1,
        df_trafic2
    ],ignore_index=True
).groupby(by='page_type').sum().reset_index()

df_trafic_final.to_csv('database/trafic.csv',index=False)

# Bounce Rate

In [12]:
df_bounce = df_cleaned.copy()

df_bounce = df_bounce.groupby(by='session').size().reset_index()

df_bounce = pd.merge(
    df_bounce,
    df_land_page,
    how='left',
    on='session'
)

df_bounce.columns = [
    'session', 
    'num_pages_session', 
    'page_type'
]

df_bounce1 = df_bounce.loc[
    df_bounce['num_pages_session'] == 1
].groupby(by='page_type')['session'].size().reset_index().rename(columns={'session':'one_page_session'})

df_bounce2 = df_bounce.groupby(by='page_type')['session'].size().reset_index().rename(columns={'session':'all_session'})

df_bounce_final = pd.concat(
    [
        df_bounce1,
        df_bounce2
    ],ignore_index=True
).groupby(by='page_type').sum().reset_index()

df_bounce_final['bounce_rate'] = df_bounce_final['one_page_session'] / df_bounce_final['all_session']

df_bounce_final.to_csv('database/bounce_rate.csv',index=False)

# Cart Abandonment

In [17]:
df_cart_abandonment = df_funnel.copy()

df_cart_abandonment = df_cart_abandonment.loc[
    df_cart_abandonment['variable'].isin(['Intent','Purchase'])
]


df_list = []
for page in df_cart_abandonment['page_type'].unique():
    df_cart_abandonment_loop = df_cart_abandonment[df_cart_abandonment['page_type']==page]

    abandon = df_cart_abandonment_loop[df_cart_abandonment_loop['variable']=='Intent']['value'].values - df_cart_abandonment_loop[df_cart_abandonment_loop['variable']=='Purchase']['value'].values

    df_cart_abandonment_loop['rate'] = abandon / df_cart_abandonment_loop['value']

    df_list.append(df_cart_abandonment_loop)

df_cart_abandonment_final = pd.concat(
    df_list,
    ignore_index=True
)

df_cart_abandonment_final = df_cart_abandonment_final[df_cart_abandonment_final['variable']=='Intent']

df_cart_abandonment_final[[
    'page_type',
    'value',
    'rate'
]]

df_cart_abandonment_final.to_csv('database/cart_abandonment.csv',index=False)

ValueError: Length of values (1) does not match length of index (2)

# Product Sold

In [44]:
df_product_sold = df_cleaned.copy()

df_intent_sessions = df_product_sold[df_product_sold['product']>0][['session','product']]

df_purchase_sessions = df_product_sold[df_product_sold['event_type']=='order'][['session']]


df_product_sold = pd.merge(
    df_intent_sessions,
    df_purchase_sessions,
    how='inner',
    on='session'
)

df_product_sold = pd.merge(
    df_product_sold,
    df_land_page,
    how='left',
    on='session'
)

df_product_sold = df_product_sold.groupby(by='page_type')['product'].size().reset_index().rename(columns={'product':'total_products_sold'})

df_product_sold.to_csv('database/total_products_sold.csv',index=True)